In [1]:
import numpy as onp
import jax.numpy as np
from jax import random
from jax import jit
from jax import vmap
from jax import lax
from jax import config
config.update("jax_enable_x64", True)


from jax_md import space, smap, energy, minimize, quantity, simulate, partition, util
from jax_md.util import f32

from collections import namedtuple

vectorize = np.vectorize

# normalize = lambda v: v / np.linalg.norm(v, axis=1, keepdims=True)

def normalize(v, v_lim=0):
    v_norm = np.linalg.norm(v, axis=1, keepdims=True)
    return np.where(v_norm > v_lim, v/v_norm, v)


from functools import partial
from simulator.utils import angle_correct, normal, align_tot, general_force_generator_TEST_1, general_force_generator_TEST_2
from simulator.render import render
from simulator.utils import ttc_potential_tot, time_to_collide, ttc_tot, ttc_potential_fn, ttc_force_tot, wall_energy, normalize_cap, wall_energy_tot
import matplotlib.pyplot as plt

class PedestrianState(namedtuple("PedestrianState", ['position', 'velocity', 'radius', 'goal_velocity'])):
    # 'group_velocity'
    def orientation(self):
        """Returns the orientation angle of all pedestrians"""
        return angle_correct(np.atan2(self.position[:, 0], self.position[:, 1]))

In [ ]:
displacement, shift = space.free()

dpos = np.array([5, 0])
rela_v = np.array([4.01, 2.99]) / 5
print(time_to_collide(dpos, rela_v, np.zeros((2, ), dtype=f32), 1.5))

In [ ]:
np.sqrt(-1)

In [ ]:
pos = np.array([[0.0, 0.0],
                [3.0, 0.0]])

velocity = np.array([[1.0, 0.0],
                     [0.0, 0.0]])

print(ttc_tot(pos, velocity, 1, displacement))

print(ttc_potential_fn(1.5, np.reshape(ttc_tot(pos, velocity, 1, displacement), shape=(4, )), 3.0))

In [ ]:
dt = np.linspace(0.2, 3., 60)
Es = ttc_potential_fn(1.5, dt, 3.0)

plt.plot(dt, Es)

In [ ]:
np.arange(0, 10)


In [ ]:
pos = np.array([[0.0, 0.0],
                [3.0, 0.0]])

velocity = np.array([[1.0, 0.0],
                     [0.0, 0.0]])

print(ttc_force_tot(pos, velocity, 1.0, displacement))
print(normalize_cap(ttc_force_tot(pos, velocity, 1.0, displacement), 5))

print(np.sum(ttc_force_tot(pos, velocity, 1.0, displacement), axis=1))
print(np.sum(normalize_cap(ttc_force_tot(pos, velocity, 1.0, displacement)), axis=1))

# print(normalize(ttc_force_tot(pos, velocity, 1.0, displacement)))
# print(np.sum(ttc_force_tot(pos, velocity, 1.0, displacement), axis=1))
# print(np.sum(normalize(ttc_force_tot(pos, velocity, 1.0, displacement)), axis=1))

In [ ]:
# @jit
# def wall_energy(pos, wall):
#    wall_start, wall_end = wall
#    wall_tangent = wall_end - wall_start
#    wall_normal = test_normalize_cap(np.array([-wall_tangent[1], wall_tangent[0]]))

#    return 1/np.dot(pos, wall_normal)

print(wall_energy(np.array([[0.0, 5.0], [3.0, 10.0]]), np.array([[0, 0], [10, 0]])))
print(quantity.force(wall_energy)(np.array([[0.0, 5.0], [3.0, 10.0]]), np.array([[0, 0], [10, 0]])))

In [ ]:
def wall_dist(pos, start, end):
    wall_len = np.dot(end - start, end - start)
    # t = max(0, min(1, np.dot(pos - start, end - start) / wall_len))
    t = np.max(np.array([0, np.min(np.array([1, np.dot(pos - start, end - start) / wall_len]), axis = 0)]), axis = 0)
    proj = start + t * (end - start)
    return displacement(pos, proj)

print(wall_dist(np.array([0.0, 3.0]), np.array([4.0, 0.0]), np.array([10.0, 0.0])))

In [ ]:
class StraightWall(namedtuple('StraightWall', ['start', 'end'])):
    pass

pos = np.array([[1.0, 0.0],
                [3.0, 0.0]])

wall1 = StraightWall(np.array([4.0, 0.0]), np.array([10.0, 0.0]))
wall2 = StraightWall(np.array([0.0, 1.0]), np.array([0.0, 9.0]))
walls = np.array([wall1, wall2])

def true_wall_energy_tot(poss, walls, displacement):
    return vmap(wall_energy_tot, (None, 0, None))(poss, walls, displacement)

true_wall_energy_tot(pos, walls, displacement)


In [6]:
paral = np.array([
    [[1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]],
    [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
])
perpen = np.array([
    [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]],
    [[0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0]]
])

force1 = general_force_generator_TEST_1(paral, perpen, 10., 10.)
force2 = general_force_generator_TEST_2(paral, perpen, 10., 10.)

test_1_init_runtime = 4.291534423828125e-06
test_2_init_runtime = 0.0023763179779052734


In [7]:
dpos = np.array([10.0, 2.0])
V_1 = np.array([1.0, 0.0])
V_2 = np.array([2.0, 0.0])
print(force1(dpos, V_1, V_2))
print(force2(dpos, V_1, V_2))

test_1_runtime = 0.042883872985839844
[-0.57126507  0.        ]
test_2_runtime = 0.0419001579284668
[-0.57126507  0.        ]


In [6]:
np.round(1.1010, 0)

Array(1., dtype=float64, weak_type=True)